In [1]:
pip install pandas requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import requests
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import numpy as np

In [4]:
"""
Fetch all the locations with a hotspot in australia
"""

MAP_KEY = "6c8a65ccb7fd9e0e7caef47ad2c3fb49"

# Bounding box for australia
bbox = [112.0, -44.0, 154.0, -10.0]

# VIIRS 375m data, last 24 hours, CSV output
# https:/firms.modaps.eosdis.nasa.gov/api/area/csv/MAP_KEY/SATELLITE or SENSOR/BOUNDINGBOX/DAYS
url = f"https://firms.modaps.eosdis.nasa.gov/api/area/csv/{MAP_KEY}/VIIRS_NOAA20_NRT/{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}/1"

# Send request
response = requests.get(url)
# Check for successful request
if response.status_code == 200:
    # Load into DataFrame
    from io import StringIO
    df = pd.read_csv(StringIO(response.text))

    # Display relevant columns
    print(df.head())
    print(df[['latitude', 'longitude', 'acq_date', 'acq_time', 'confidence']])
else:
    print(f"Failed to fetch data: {response.status_code}")

   latitude  longitude  bright_ti4  scan  track    acq_date  acq_time  \
0 -43.34805  146.91154      354.01  0.60   0.53  2025-05-12       325   
1 -43.34264  146.91872      340.35  0.60   0.53  2025-05-12       325   
2 -43.34224  146.91891      338.16  0.60   0.53  2025-05-12       325   
3 -42.78127  146.90149      345.14  0.60   0.53  2025-05-12       325   
4 -42.77414  146.93643      325.38  0.59   0.53  2025-05-12       325   

  satellite instrument confidence version  bright_ti5     frp daynight  
0       N20      VIIRS          n  2.0NRT      290.67  138.46        D  
1       N20      VIIRS          n  2.0NRT      288.18  138.46        D  
2       N20      VIIRS          n  2.0NRT      285.02   47.29        D  
3       N20      VIIRS          n  2.0NRT      286.98    9.42        D  
4       N20      VIIRS          n  2.0NRT      285.79    3.62        D  
      latitude  longitude    acq_date  acq_time confidence
0    -43.34805  146.91154  2025-05-12       325          n
1    

In [16]:
"""
Fetch the weather for all the locations
From FIRM's high confidence hotspots first.
"""

url = "https://api.open-meteo.com/v1/forecast"
for row in df.head(5).iterrows():
    params = {
        'latitude': row[1].latitude,
        'longitude': row[1].longitude,
        "hourly": "temperature_2m,relative_humidity_2m,precipitation,soil_moisture_0_1cm,shortwave_radiation,windspeed_10m",
        "timezone": "auto",
    }
    response = requests.get(url, params=params)
    print(params["latitude"], params['longitude'])
    # Parse the response
    if response.status_code == 200:
        data = response.json()
        hourly = data.get("hourly", {})
        
        # Print some values (e.g., first time step for each variable)
        print("Time:", hourly["time"][0])
        print("Temperature (°C):", hourly["temperature_2m"][0])
        print("Relative Humidity (%):", hourly["relative_humidity_2m"][0])
        print("Precipitation (mm):", hourly["precipitation"][0])
        print("Soil Water Content (m³/m³):", hourly["soil_moisture_0_1cm"][0])
        print("Solar Radiation (W/m²):", hourly["shortwave_radiation"][0])
        print("Wind Speed (m/s):", hourly["windspeed_10m"][0])
    else:
        print("Failed to fetch data. Status code:", response.status_code)
        print(response.text)

-43.34805 146.91154
Time: 2025-05-12T00:00
Temperature (°C): 8.4
Relative Humidity (%): 91
Precipitation (mm): 0.0
Soil Water Content (m³/m³): 0.118
Solar Radiation (W/m²): 0.0
Wind Speed (m/s): 7.4
-43.34264 146.91872
Time: 2025-05-12T00:00
Temperature (°C): 8.2
Relative Humidity (%): 91
Precipitation (mm): 0.0
Soil Water Content (m³/m³): 0.118
Solar Radiation (W/m²): 0.0
Wind Speed (m/s): 7.4
-43.34224 146.91891
Time: 2025-05-12T00:00
Temperature (°C): 8.1
Relative Humidity (%): 91
Precipitation (mm): 0.0
Soil Water Content (m³/m³): 0.118
Solar Radiation (W/m²): 0.0
Wind Speed (m/s): 7.4
-42.78127 146.90149
Time: 2025-05-12T00:00
Temperature (°C): 8.1
Relative Humidity (%): 71
Precipitation (mm): 0.0
Soil Water Content (m³/m³): 0.144
Solar Radiation (W/m²): 0.0
Wind Speed (m/s): 3.4
-42.77414 146.93643
Time: 2025-05-12T00:00
Temperature (°C): 7.9
Relative Humidity (%): 71
Precipitation (mm): 0.0
Soil Water Content (m³/m³): 0.144
Solar Radiation (W/m²): 0.0
Wind Speed (m/s): 3.4


In [20]:
"""
Class for ML model
"""

class RegressionNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [24]:
data_df = pd.read_csv("flattened_wildfire_data.csv")
X = data_df.drop(["Mean_confidence"], axis=1).values

In [26]:
input_dim = X.shape[1] 
model = RegressionNN(input_dim)
model.load_state_dict(torch.load("wildfire_nn_weights1.pth"))
model.eval()

RegressionNN(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [28]:
# Refit scalers if not saved
df = pd.read_csv("flattened_wildfire_data.csv")
df = df.select_dtypes(include=["number"]).dropna()

X = df.drop("Mean_confidence", axis=1).values
scaler_X = StandardScaler().fit(X)
scaler_y = StandardScaler().fit(df["Mean_confidence"].values.reshape(-1, 1))

# Example new data input (same number of features as training data)
new_input = np.array([[feature1, feature2, ..., featureN]])
new_input_scaled = scaler_X.transform(new_input)

# Convert to tensor
new_input_tensor = torch.tensor(new_input_scaled, dtype=torch.float32)


NameError: name 'feature1' is not defined